In [27]:
import numpy as np
from scipy.stats import multivariate_normal
import matplotlib.pyplot as plt

In [2]:
# MESH VARIABLES
xgrid = 100
ygrid = 100
zgrid = 200

In [24]:
def create_mesh_grid():
    xmax = xgrid - 1
    ymax = ygrid - 1
    zmax = zgrid - 1
    x, y, z = np.mgrid[0:1:(xmax*1j), 0:1:(ymax*1j), 0:1:(zmax*1j)]

    return x, y, z

In [69]:
def gauss3d_o3_profile():
    x, y, z = create_mesh_grid()
    # Need an (N, 2) array of (x, y) pairs.
    xyz = np.column_stack([x.flat, y.flat, z.flat])

    mu = np.array([.5, .5, 0])

    sigma = np.array([.25, .25, .15])
    covariance = np.diag(sigma**2)

    val = multivariate_normal.pdf(xyz, mean=mu, cov=covariance)

    # Reshape back to a (30, 30) grid.
    val = val.reshape(x.shape)

    # Normalize max to 100 (ppmv) and transpose so orientation is correct (z up)
    val = val.T*(100/val.max())

    # Assign minimum value via ambient concentration populated in array by WRF-Chem
    # prior to modification
    val[val<0.03] = 0.03

    print(val.shape)

    new_o3_vals = np.ma.array(np.array([val]), mask=False, dtype='float32')

    return new_o3_vals

In [72]:
def checkerboard_profile():
    xrange, yrange = xgrid-1, ygrid-1
    epsilon = 0.001
    Ax, Ay = 1, 1
    fx, fy = 1, 1
    k = fx*2*np.pi/xrange #wavenumber 2pi / L
    m = fy*2*np.pi/yrange
    x=np.arange(xrange)
    y=np.arange(yrange)
    X,Y=np.meshgrid(x,y)
    phi=Ax*np.sin(k*X+epsilon)*Ay*np.sin(m*Y+epsilon)

    phi_star = phi.copy()

    phi_star[phi_star > 0] = 1
    phi_star[phi_star <= 0] = 0

    mesh = np.zeros((xgrid-1, ygrid-1, zgrid-1))
    mesh[:, :, 0] = phi_star

    # transpose so z, y, x
    mesh = mesh.T

    mesh = np.ma.array(np.array([mesh]), mask=False, dtype='float32')

    return mesh


In [70]:
o3 = gauss3d_o3_profile()

(199, 99, 99)


In [68]:
o3[0].data.shape

(199, 99, 99)

In [73]:
check_mesh = checkerboard_profile()